# **Advent of Code 2022**

In [2]:
import numpy as np
import pandas as pd
from functools import reduce

import warnings
warnings.filterwarnings("ignore")

## **1. Advent**

In [3]:
l = [i.split("\n") for i in open("inputs22/1").read().split("\n\n")]
s = [sum([int(i) for i in sl]) for sl in l]
print(f"Task 1: {max(s)}, Task 2: {sum(sorted(s)[-3:])}")

Task 1: 69795, Task 2: 208437


In [23]:
d = pd.read_csv("inputs22/2", header=None, sep=" ")

def f1(opp, you):
    match (opp, you):
        case ("A", "X"): return 3 + 1
        case ("B", "X"): return 0 + 1
        case ("C", "X"): return 6 + 1
        case ("A", "Y"): return 6 + 2
        case ("B", "Y"): return 3 + 2
        case ("C", "Y"): return 0 + 2
        case ("A", "Z"): return 0 + 3
        case ("B", "Z"): return 6 + 3
        case ("C", "Z"): return 3 + 3

def f2(opp, you):
    match (opp, you):
        case ("A", "X"): return 0 + 3
        case ("B", "X"): return 0 + 1
        case ("C", "X"): return 0 + 2
        case ("A", "Y"): return 3 + 1
        case ("B", "Y"): return 3 + 2
        case ("C", "Y"): return 3 + 3
        case ("A", "Z"): return 6 + 2
        case ("B", "Z"): return 6 + 3
        case ("C", "Z"): return 6 + 1

d.apply(lambda row: f2(row[0], row[1]), axis=1).sum()xw

10116

In [19]:
d

,0,1
0,A,Y
1,B,X
2,C,Z
